In [1]:
# HACK: use project root as the working directory 
from pathlib import Path

while Path.cwd().name != 'language-model-toxicity':
    %cd ..

/home/msap/research/language-model-toxicity


In [11]:
import sys, os, json
import pandas as pd

sys.path.append(os.path.join(os.path.expanduser("~"),"resources/"))
from stats import statstools
from idp import idp

from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)

from utils.constants import DATA_DIR, SHARED_DIR, PERSPECTIVE_API_ATTRIBUTES_LOWER
generations_dir = SHARED_DIR / 'generations' / 'prompt_n_50percent_toxicity_quartiles_25k_subsample'
dialectFile = DATA_DIR / "prompts/prompts_n_50percent_toxicity_quartiles_25k_subsample.dialects.csv"
gpt2_file = generations_dir / 'gpt2' / 'prompt_n_50percent_toxicity_quartiles_25k_subsample_gpt2.jsonl'

In [3]:
# Reading dialect file
df = pd.read_csv(dialectFile)
df.columns

Index(['filename', 'begin', 'end', 'prompt.text', 'prompt.profanity',
       'prompt.sexually_explicit', 'prompt.identity_attack',
       'prompt.flirtation', 'prompt.threat', 'prompt.insult',
       'prompt.severe_toxicity', 'prompt.toxicity', 'continuation.text',
       'continuation.severe_toxicity', 'continuation.toxicity',
       'continuation.profanity', 'continuation.sexually_explicit',
       'continuation.identity_attack', 'continuation.flirtation',
       'continuation.threat', 'continuation.insult', 'prompt.aav',
       'prompt.hispanic', 'prompt.other', 'prompt.white', 'prompt.dial_argmax',
       'continuation.aav', 'continuation.hispanic', 'continuation.other',
       'continuation.white', 'continuation.dial_argmax'],
      dtype='object')

# Prompt/Continuations: 
## Correlations with AAE
Double checking that correlations between prompts continuations

In [4]:
out = {}
for c in ["prompt","continuation"]:
    d = statstools.correlSummary(df,f"{c}.aav",f"{c}.toxicity")
    out[c] = d
pd.DataFrame(out).T

,n,r,p
prompt,99464.0,0.155377,0.0
continuation,99329.0,0.208111,0.0


# Generations
## Extracting dialect categories

In [5]:
sys.path.append(os.path.join(os.path.expanduser("~"),"tools/"))
import getDialect as dialExtract

In [25]:
genData = {(i,j):g for i,l in tqdm(enumerate(open(gpt2_file)))
           for j, g in enumerate(json.loads(l)["generations"])}# if i < 1000}

In [26]:
genDf = pd.DataFrame.from_dict(genData,orient="index")
genDf.index.names = ["promptId","genId"]
genDf.reset_index(inplace=True)
print(genDf.shape)

(2500000, 11)


In [34]:
genDfSample = genDf.sample(500000)

In [37]:
tqdm.pandas(ascii=True)
feats = genDf["text"].progress_apply(dialExtract.countDialectWords)
genDf[feats.columns] = feats

/home/msap/anaconda3/envs/lm-toxicity/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|##########| 2500000/2500000 [1:00:13<00:00, 691.92it/s]


## Correlations with AAE

In [38]:
pd.Series(statstools.correlSummary(genDf,"aav","toxicity")).to_frame(name="gens").T

,n,r,p
gens,2485237.0,0.141928,0.0


In [36]:
pd.Series(statstools.correlSummary(genDfSample,"aav","toxicity")).to_frame(name="gens").T

,n,r,p
gens,497091.0,0.144867,0.0
